In [1]:
import torch
import numpy as np
from sklearn.datasets import load_files
from glob import glob
import cv2 # type: ignore
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os

In [2]:
#狗的数据集
def load_data(path): 
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = torch.nn.functional.one_hot(torch.tensor(np.array(data['target'])).long(),133)  # 将类别转换成数组。
    print(dog_targets.shape)
    return dog_files,dog_targets
train_files,train_targets = load_data('./dogImages/train')
valid_files,valid_targets = load_data('./dogImages/valid')  
test_files,test_targets = load_data('./dogImages/test')
dog_names = [item[20:-1] for item in sorted(glob("./dogImages/train/*/"))]


torch.Size([6680, 133])
torch.Size([835, 133])
torch.Size([836, 133])


In [3]:
#人脸的数据集
random.seed(8675309)
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)  #type:ignore

In [4]:
#人脸检测模型加载
face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_alt.xml') # type: ignore


In [5]:
def face_detector(img_path):
    img = cv2.imread(img_path) #type:ignore
    if img is not None:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #type:ignore
        faces = face_cascade.detectMultiScale(gray)
    
    #圈出模型识别出的人脸在图中位置。
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1) # type: ignore
    cv_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # type: ignore
    plt.imshow(cv_rgb)
    plt.show()
    plt.close()
    return faces
    

In [ ]:
face_detection = []
for i in human_files:
    for j in os.listdir(i):
        face_detection.append(face_detector(os.path.join(i,j)))

In [6]:
human_files_short = human_files[:100]
dog_files_short = train_files[:100]

In [7]:
def model_ignores(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #type:ignore
    detect_face = len(face_cascade.detectMultiScale(gray)) # type: ignore
    return detect_face>0

In [8]:
error_num=0
for i in human_files_short:
    for j in os.listdir(i):
        detect_face = model_ignores(os.path.join(i,j))
        if(detect_face==False):
            error_num+=1;
            print("human_files[{}] detect human face fail".format(os.path.join(i,j)))
print("error check number:{}".format(error_num))
error_num=0
for i in dog_files_short:
    detect_face = model_ignores(i)
    if(detect_face==True):
        error_num+=1
        print("dog_files[{}] is detectted as human face".format(i))
print("error check number:{}".format(error_num))

human_files[lfw\lfw\Taia_Balk\Taia_Balk_0001.jpg] detect human face fail
error check number:1
dog_files[./dogImages/train\095.Kuvasz\Kuvasz_06442.jpg] is detectted as human face
dog_files[./dogImages/train\099.Lhasa_apso\Lhasa_apso_06646.jpg] is detectted as human face
dog_files[./dogImages/train\009.American_water_spaniel\American_water_spaniel_00628.jpg] is detectted as human face
dog_files[./dogImages/train\057.Dalmatian\Dalmatian_04023.jpg] is detectted as human face
dog_files[./dogImages/train\096.Labrador_retriever\Labrador_retriever_06474.jpg] is detectted as human face
dog_files[./dogImages/train\106.Newfoundland\Newfoundland_06989.jpg] is detectted as human face
dog_files[./dogImages/train\117.Pekingese\Pekingese_07559.jpg] is detectted as human face
dog_files[./dogImages/train\039.Bull_terrier\Bull_terrier_02805.jpg] is detectted as human face
dog_files[./dogImages/train\097.Lakeland_terrier\Lakeland_terrier_06516.jpg] is detectted as human face
dog_files[./dogImages/train\05